In [47]:
def average(num_1, num_2):
    numbers = num_1.split(' & ')
    numbers.extend(num_2.split(' & '))
    numbers = set([float(num.strip()) for num in numbers])
    summation = sum(list(numbers))
    average = summation / len(numbers)
    return average

In [46]:
from numpy import nan
import pandas
import json
import os
import re

empty = [nan, None, ' ', '']

master_file = pandas.read_csv('2021-09-04_master_TECR_2.csv')
# combine the ionic strength columns
for index, value in master_file[['Ionic strength [mol / dm^3]', 'Ionic strength [mol / kg]']].iterrows():
    if value['Ionic strength [mol / kg]'] in empty:
        master_file.at[index, 'Ionic strength [mol / kg]'] = value['Ionic strength [mol / dm^3]']
    elif value['Ionic strength [mol / kg]'] not in empty:
        if value['Ionic strength [mol / dm^3]'] not in empty:
            if value['Ionic strength [mol / dm^3]'] != value['Ionic strength [mol / kg]']:
                print(value['Ionic strength [mol / dm^3]'], '\t', value['Ionic strength [mol / kg]'])
                master_file.at[index, 'Ionic strength [mol / kg]'] = average(value['Ionic strength [mol / kg]'], value['Ionic strength [mol / dm^3]'])

for index, row in master_file.iterrows():
    # open a new JSON template
    json_template = json.load(open('template.json'))
    # define components of the JSON
    for key, value in json_template.items():
        if key == 'curator':
            value['orcid'] = 'https://orcid.org/0000-0002-7288-535X'
            value['name'] = 'Andrew Philip Freiburger' 
        elif key == 'CuratedMeasurement':
            # define the reference content
            value['reference']['pmid'] = row['PMID']
            value['reference']['doi'] = row['DOI']
            value['reference']['tecrdb_string'] = row['Reference:']
            value['reference']['tecrdb_id'] = row['Reference ID:']
            
            # define the reaction 
            value['representative_reaction']['name'] = row['Enzyme:']
            value['representative_reaction']['KEGG'] = row['KEGG Reaction:']
            value['representative_reaction']['CID'] = row['CID Reaction:']
            
#             print(row['Reaction:'])
            if not re.search('=', row['Reaction:']):
                value['representative_reaction']['stoichiometry'] = row['Reaction:']
            else:
                reactants, products = row['Reaction:'].split('=')
                reactants = reactants.split(' + ')
                reactants_dict = {}
                for reactant in reactants:
                    coef = re.search('\d\s', reactant)
                    if coef is None:
                        coef = 1
                    else:
                        coef = coef.group()
                    reactants_dict[reactant] = coef

                products = products.split(' + ')
                products_dict = {}
                for product in products:
                    coef = re.search('\d\s', product)
                    if coef is None:
                        coef = 1
                    else:
                        coef = coef.group()
                    products_dict[product] = coef
                
                value['representative_reaction']['stoichiometry']['reactants'] = reactants_dict
                value['representative_reaction']['stoichiometry']['products'] = products_dict
            
            # define the measurement data
            value['equilibriumConstant'] = row['Keq']
            value['hydrogenPotential'] = row['pH ']
            value['temperature'] = row['T [K]']
            value['ionicStrength'] = row['Ionic strength [mol / kg]']
            pmg = re.search('(\d+.?\d+) = -log\[Mg\+2\]', row['Experimental conditions'])
            if pmg is not None:
                value['magnesiumPotential'] = pmg.group()
                
                
            # define comments
            
    # export the datum JSON
    os.mkdir('./datum_points')
    count = 0
    ec = row['EC Value:']
    reference = re.sub('(/)', '-', row['Reference ID:'])
    export_name = '_'.join([reference, ec, str(count)])
    while os.path.exists(f'./datum_points/{export_name}.json'):
        count += 1
        export_name = '_'.join([reference, ec, str(count)])
    
    with open(f'./datum_points/{export_name}.json', 'w') as out:
        json.dump(json_template, out, indent = 4)

1.53 	 0.063
0.063 	 1.53
0.253 	 0.297
0.297 	 0.253
0.727 	 0.726
0.726 	 0.727
0.726 	 0.727
0.727 	 0.726
0.697 	 0.703
0.703 	 0.697
0.703 	 0.697
0.697 	 0.703
0.082 	 0.081
0.081 	 0.082
0.081 	 0.082
0.082 	 0.081
0.56 & 0.14 	 0.56
0.56 	 0.56 & 0.14
0.046 & 0.085 	 0.046
0.046 	 0.046 & 0.085
0.032 & 0.046 	 0.032
0.032 	 0.032 & 0.046
0.038 & 0.032 	 0.038
0.038 	 0.038 & 0.032
0.085 & 0.038 	 0.085
0.085 	 0.085 & 0.038
0.1 & 0.07 	 0.1
0.1 	 0.1 & 0.07
0.07 & 0.1 	 0.07
0.07 	 0.07 & 0.1
0.07 & 0.065 	 0.07
0.07 	 0.07 & 0.065
0.284 	 0.283
0.283 	 0.284
0.263 	 0.268
0.268 	 0.263
0.592 	 0.11
0.11 	 0.592
0.723 	 0.172
0.172 	 0.723
0.013 	 0.012
0.012 	 0.013
0.454 	 0.093
0.093 	 0.454
1.67 	 0.594
0.594 	 1.67
0.11 	 0.592
0.592 	 0.11
0.093 	 0.454
0.454 	 0.093
0.172 	 0.723
0.723 	 0.172
0.012 	 0.013
0.013 	 0.012
0.594 	 1.67
1.67 	 0.594
0.283 	 0.284
0.284 	 0.283
0.268 	 0.263
0.263 	 0.268
1 	 1.0
1.0 	 1
1 	 1.0
1.0 	 1


# Brainstorming

In [10]:
import pandas
master_file = pandas.read_csv('2021-09-04_master_TECR_2.csv')
for index in master_file[['Ionic strength [mol / dm^3]', 'Ionic strength [mol / kg]']].iterrows():
    print(index)

(0, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 0, dtype: object)
(1, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1, dtype: object)
(2, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 2, dtype: object)
(3, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3, dtype: object)
(4, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4, dtype: object)
(5, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 5, dtype: object)
(6, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 6, dtype: object)
(7, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 7, dtype: object)
(8, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 8, dtype: object)
(9, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 9, dtype: object)
(10, Ionic strength [mol / dm^3]     
Ionic strength [mol / 

Name: 517, dtype: object)
(518, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 518, dtype: object)
(519, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 519, dtype: object)
(520, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 520, dtype: object)
(521, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 521, dtype: object)
(522, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 522, dtype: object)
(523, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 523, dtype: object)
(524, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 524, dtype: object)
(525, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 525, dtype: object)
(526, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 526, dtype: object)
(527, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 527, dtype: ob

Name: 1038, dtype: object)
(1039, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1039, dtype: object)
(1040, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1040, dtype: object)
(1041, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1041, dtype: object)
(1042, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1042, dtype: object)
(1043, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1043, dtype: object)
(1044, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1044, dtype: object)
(1045, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1045, dtype: object)
(1046, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1046, dtype: object)
(1047, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1047, dtype: object)
(1048, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       


Name: 1548, dtype: object)
(1549, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1549, dtype: object)
(1550, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1550, dtype: object)
(1551, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1551, dtype: object)
(1552, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1552, dtype: object)
(1553, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1553, dtype: object)
(1554, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1554, dtype: object)
(1555, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1555, dtype: object)
(1556, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1556, dtype: object)
(1557, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 1557, dtype: object)
(1558, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       


Name: 2054, dtype: object)
(2055, Ionic strength [mol / dm^3]    0.25
Ionic strength [mol / kg]          
Name: 2055, dtype: object)
(2056, Ionic strength [mol / dm^3]    0.25
Ionic strength [mol / kg]          
Name: 2056, dtype: object)
(2057, Ionic strength [mol / dm^3]    0.25
Ionic strength [mol / kg]          
Name: 2057, dtype: object)
(2058, Ionic strength [mol / dm^3]    0.25
Ionic strength [mol / kg]          
Name: 2058, dtype: object)
(2059, Ionic strength [mol / dm^3]    0.25
Ionic strength [mol / kg]          
Name: 2059, dtype: object)
(2060, Ionic strength [mol / dm^3]    0.25
Ionic strength [mol / kg]          
Name: 2060, dtype: object)
(2061, Ionic strength [mol / dm^3]    0.25
Ionic strength [mol / kg]          
Name: 2061, dtype: object)
(2062, Ionic strength [mol / dm^3]    0.25
Ionic strength [mol / kg]          
Name: 2062, dtype: object)
(2063, Ionic strength [mol / dm^3]    0.25
Ionic strength [mol / kg]          
Name: 2063, dtype: object)
(2064, Ionic streng

...

Name: 3066, dtype: object)
(3067, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3067, dtype: object)
(3068, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3068, dtype: object)
(3069, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3069, dtype: object)
(3070, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3070, dtype: object)
(3071, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3071, dtype: object)
(3072, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3072, dtype: object)
(3073, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3073, dtype: object)
(3074, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3074, dtype: object)
(3075, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3075, dtype: object)
(3076, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       


Name: 3575, dtype: object)
(3576, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3576, dtype: object)
(3577, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3577, dtype: object)
(3578, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3578, dtype: object)
(3579, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3579, dtype: object)
(3580, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3580, dtype: object)
(3581, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3581, dtype: object)
(3582, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3582, dtype: object)
(3583, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3583, dtype: object)
(3584, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 3584, dtype: object)
(3585, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       


Name: 4089, dtype: object)
(4090, Ionic strength [mol / dm^3]    0.29
Ionic strength [mol / kg]      0.29
Name: 4090, dtype: object)
(4091, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4091, dtype: object)
(4092, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4092, dtype: object)
(4093, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4093, dtype: object)
(4094, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4094, dtype: object)
(4095, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4095, dtype: object)
(4096, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4096, dtype: object)
(4097, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4097, dtype: object)
(4098, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4098, dtype: object)
(4099, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]  

Name: 4590, dtype: object)
(4591, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4591, dtype: object)
(4592, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4592, dtype: object)
(4593, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4593, dtype: object)
(4594, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4594, dtype: object)
(4595, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4595, dtype: object)
(4596, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4596, dtype: object)
(4597, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4597, dtype: object)
(4598, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4598, dtype: object)
(4599, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 4599, dtype: object)
(4600, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       


Name: 5087, dtype: object)
(5088, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 5088, dtype: object)
(5089, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 5089, dtype: object)
(5090, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 5090, dtype: object)
(5091, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 5091, dtype: object)
(5092, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 5092, dtype: object)
(5093, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 5093, dtype: object)
(5094, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 5094, dtype: object)
(5095, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 5095, dtype: object)
(5096, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       
Name: 5096, dtype: object)
(5097, Ionic strength [mol / dm^3]     
Ionic strength [mol / kg]       


In [5]:
reactants, products = 'CHB_15635 + CHB_74819 = CHB_15377 + CHB_15570 + CHB_1989'.split('=')